This notebook loads the raw DP1 catalog, performs star-galaxy separation, then de-reddens the magnitudes.

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.spatial import ConvexHull
import pandas as pd

In [ ]:
# Load the raw catalog
raw = Table.read("../data/dp1_ecdfs_raw.fits")
raw

## Star-galaxy separation

In [ ]:
ref_flux_ratio = []
ref_flux_ratio_err = []
for i in range(len(raw)):
    # Grab values for row
    band = raw["refBand"][i]
    psfFlux = raw[f"{band}_psfFlux"][i]
    psfFluxErr = raw[f"{band}_psfFluxErr"][i]
    cModelFlux = raw[f"{band}_cModelFlux"][i]
    cModelFluxErr = raw[f"{band}_cModelFluxErr"][i]

    # Calculate flux ratio in reference band
    ref_flux_ratio.append(psfFlux / cModelFlux)

    # Calculate error on the flux ratio
    # uses standard error propagation, assuming two flux types are uncorrelated
    # bad assumption, but this matches what Alex DW did
    with np.errstate(over="ignore"):
        err_ratio = np.sqrt(
            (psfFluxErr / psfFlux) ** 2 + (cModelFluxErr / cModelFlux) ** 2
        )

    ref_flux_ratio_err.append(ref_flux_ratio[-1] * err_ratio)

ref_flux_ratio = np.array(ref_flux_ratio)
ref_flux_ratio_err = np.array(ref_flux_ratio_err)

# Calculate the probability that the flux ratio is consistent with the ratio cut
ratio_cut = 0.985  # Cut used in DP1 default star-galaxy separation
p_star = 1 - norm.cdf((ratio_cut - ref_flux_ratio) / ref_flux_ratio_err)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=150)

xlim = (15, 26.7)

# Define the cut on p_star. I.e. objects with p_star > p_star_cut are stars
p_star_cut = 0.65

# Galaxy distribution in red
ax.hexbin(
    raw["i_cModelMag"][p_star < p_star_cut],
    p_star[p_star < p_star_cut],
    gridsize=200,
    norm="log",
    cmap="Reds",
    rasterized=True,
)

# Star distribution in blue
ax.hexbin(
    raw["i_cModelMag"][p_star > p_star_cut],
    p_star[p_star > p_star_cut],
    gridsize=200,
    norm="log",
    cmap="Blues",
    rasterized=True,
)

# Show the cut
ax.axhline(p_star_cut, c="r", ls="--", lw=1)

# Plot settings
ax.set(
    xlabel="$i$-band cModel Mag",
    ylabel="$p_\\text{star}$",
    xlim=xlim,
    ylim=(-0.01, 1.01),
)

fig.savefig("../figures/p_star_cut.pdf", bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 1.9), dpi=200)

ylim = (-0.0, 1.06)

# Galaxies from flux ratio cut
axes[0].hexbin(
    raw["i_cModelMag"][ref_flux_ratio < ratio_cut],
    ref_flux_ratio[ref_flux_ratio < ratio_cut],
    extent=(*xlim, *ylim),
    gridsize=200,
    norm="log",
    cmap="Reds",
    rasterized=True,
)

# Stars from flux ratio cut
axes[0].hexbin(
    raw["i_cModelMag"][ref_flux_ratio > ratio_cut],
    ref_flux_ratio[ref_flux_ratio > ratio_cut],
    extent=(*xlim, *ylim),
    gridsize=200,
    norm="log",
    cmap="Blues",
    rasterized=True,
)

axes[0].set(
    xlabel="$i$-band cModel mag",
    ylabel="Ref. flux ratio",
    xlim=xlim,
    ylim=ylim,
    title="Flux ratio cut",
)

# Galaxies from p_star cut
axes[1].hexbin(
    raw["i_cModelMag"][p_star < p_star_cut],
    ref_flux_ratio[p_star < p_star_cut],
    extent=(*xlim, *ylim),
    gridsize=200,
    norm="log",
    cmap="Reds",
    alpha=0.5,
    rasterized=True,
)

# Stars from p_star cut
axes[1].scatter(
    raw["i_cModelMag"][p_star > p_star_cut],
    ref_flux_ratio[p_star > p_star_cut],
    color="C0",
    marker=".",
    s=0.05,
    rasterized=True,
)

axes[1].set(
    xlabel="$i$-band cModel mag",
    ylabel="Ref. flux ratio",
    xlim=xlim,
    ylim=ylim,
    title="$p_\\text{star}$ cut",
)

# Histogram of stars
hist_settings = dict(histtype="step", range=xlim, bins=100, density=False)
axes[2].hist(
    raw["i_cModelMag"][ref_flux_ratio > ratio_cut],
    color="C0",
    **hist_settings,
    label="Flux ratio cut",
)
axes[2].hist(
    raw["i_cModelMag"][p_star > p_star_cut],
    color="C3",
    **hist_settings,
    label="$p_\\text{star}$ cut",
)
axes[2].set(
    yscale="log",
    xlabel="$i$-band cModel mag",
    ylabel="Number of stars",
)
axes[2].legend(handlelength=1, fontsize=8, frameon=False, handletextpad=0.5)

fig.subplots_adjust(wspace=0.4)
fig.savefig("../figures/star_galaxy_separation.pdf", bbox_inches="tight")

In [ ]:
# Apply star-galaxy separation
# galaxies = raw[p_star < p_star_cut] # Custom star-galaxy separation
galaxies = raw[ref_flux_ratio < ratio_cut]  # Default star-galaxy separation

# Apply magnitude cut
galaxies = galaxies[galaxies["i_cModelMag"] > 23].to_pandas()

## De-redden

Use SFD map

In [ ]:
# De-redden magnitudes
band_a_ebv = dict(
    u=4.81,
    g=3.64,
    r=2.70,
    i=2.06,
    z=1.58,
    y=1.31,
)
for col in galaxies.columns:
    if col.endswith("Mag"):
        band = col[0]  # Assumes band is first character of column name
        galaxies[col] -= galaxies["ebv"] * band_a_ebv[band]
    elif col.endswith("Flux"):
        band = col[0]  # Assumes band is first character of column name
        galaxies[col] *= 10 ** (galaxies["ebv"] * band_a_ebv[band] / 2.5)

## Cross-match with ASTRODEEP

In [ ]:
# Coord object for DP1 galaxies
dp1_coords = SkyCoord(ra=galaxies["coord_ra"], dec=galaxies["coord_dec"], unit="deg")

# Load ASTRODEEP data
astrodeep_phys = pd.read_csv("../data/ASTRODEEP-GS43_phys.cat", delim_whitespace=True)
astrodeep_phys = astrodeep_phys[["#ID", "z_best", "z_LePhare", "z_EAzY", "z_z-phot", "zspec_survey"]]
astrodeep_phys = astrodeep_phys.rename({"#ID": "astrodeep_ID"})

astrodeep_phot = pd.read_csv("../data/ASTRODEEP-GS43_phot.cat", delim_whitespace=True)

# Coord object for ASTRODEEP
astrodeep_coords = SkyCoord(
    ra=astrodeep_phot["RA"],
    dec=astrodeep_phot["DEC"],
    unit="deg",
)

# Match DP1 and astrodeep
idx, d2d, _ = dp1_coords.match_to_catalog_sky(astrodeep_coords)
d2d = d2d.to(u.arcsec).value

# Set the max separation distance
MAX_SEP = 1.0  # arcsecs

# Replace large separations with idx=-1
idx[d2d > MAX_SEP] = -1

# Replace duplicates with -1
elements, counts = np.unique(idx, return_counts=True)
idx[np.isin(idx, elements[counts > 1])] = -1

# Match the spec-z catalog into ComCam
z_match = astrodeep_phys.reset_index(drop=True)
z_match.loc[-1] = [np.nan, np.nan, np.nan, np.nan, np.nan, "-"]  # Add empty row for no match
z_match = z_match.loc[idx].reset_index(drop=True)

combined = pd.concat([galaxies, z_match], axis=1)

print(f"{int(np.isfinite(combined["z_best"]).sum())} galaxies have spec-z's")

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=150)

ax.hist(d2d, bins=np.geomspace(1e-2, 200))
ax.set(
    xscale="log",
    xlabel="Distance to nearest match (arcsec)",
    ylabel="Number of matches",
    xlim=(4e-2, 200),
    title="Cross-matching DP1 and ASTRODEEP",
)
ax.axvline(MAX_SEP, c="r", ls="--")

fig.savefig("../figures/crossmatch_astrodeep.pdf", bbox_inches="tight")

## Cross-match with Euclid

In [ ]:
# Coord object for DP1 galaxies
dp1_coords = SkyCoord(ra=galaxies["coord_ra"], dec=galaxies["coord_dec"], unit="deg")

# Load Euclid data
euclid = Table.read("../data/euclid_ecdfs.fits").to_pandas()

# Convert fluxes from microJanskys to AB mags
for band in ["vis"] + list("yjh"):
    flux = euclid[f"flux_{band}_unif"]
    err = euclid[f"fluxerr_{band}_unif"]
    euclid[f"e{band[0]}_unifMag"] = -2.5 * np.log10(flux) + 23.9
    euclid[f"e{band[0]}_unifMagErr"] = 2.5 / np.log(10) * err / np.clip(flux, 0, None)

# Coord object for Euclid
euclid_coords = SkyCoord(
    ra=euclid["right_ascension"],
    dec=euclid["declination"],
    unit="deg",
)

# Match DP1 and Euclid
idx, d2d, _ = dp1_coords.match_to_catalog_sky(euclid_coords)
d2d = d2d.to(u.arcsec).value

# Set the max separation distance
MAX_SEP = 0.8  # arcsecs

# Replace large separations with idx=-1
idx[d2d > MAX_SEP] = -1

# Replace duplicates with -1
elements, counts = np.unique(idx, return_counts=True)
idx[np.isin(idx, elements[counts > 1])] = -1

# Down-select columns
euclid = euclid[["euclid_id"] + [col for col in euclid.columns if "Mag" in col]]

# Match the spec-z catalog into ComCam
euclid_match = euclid.reset_index(drop=True)
euclid_match.loc[-1] = [np.nan] * len(euclid.columns)  # Add empty row for no match
euclid_match = euclid_match.loc[idx].reset_index(drop=True)

combined = pd.concat([combined, euclid_match], axis=1)

print(f"{int(np.isfinite(combined["ev_unifMag"]).sum())} galaxies have spec-z's")

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=150)

ax.hist(d2d, bins=np.geomspace(3e-3, 30))
ax.set(
    xscale="log",
    xlabel="Distance to nearest match (arcsec)",
    ylabel="Number of matches",
    xlim=(3e-3, 30),
    title="Cross-matching DP1 and Euclid",
)
ax.axvline(MAX_SEP, c="r", ls="--")

fig.savefig("../figures/crossmatch_euclid.pdf", bbox_inches="tight")

## Cutout central region

In [ ]:
# Calculate the effective 5-sigma depth in the i-band
i_snr = combined.i_cModelFlux / combined.i_cModelFluxErr
i5 = combined.i_cModelMag - 2.5 * np.log10(5 / i_snr)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 2.0), dpi=150)

# Plot effective depth using hexbin
hb = ax1.hexbin(
    combined.coord_ra,
    combined.coord_dec,
    C=i5,
    reduce_C_function=np.mean,
    gridsize=300,
    rasterized=True,
)

# Add colorbar
cbar = fig.colorbar(hb, ax=ax1, shrink=0.92)
cbar.set_label("$i$-band 5$\\sigma$ depth")

ax1.set(
    aspect="equal",
    xlabel="RA (deg)",
    ylabel="Dec (deg)",
)
ax1.invert_xaxis()

# Plot redshift sample
z_cat = combined[np.isfinite(combined.z_best)]
# Compute convex hull of the redshift sample region
points = np.column_stack((z_cat.coord_ra, z_cat.coord_dec))
hull = ConvexHull(points)
hull_path = np.append(hull.vertices, hull.vertices[0])  # close the polygon
ax1.plot(
    points[hull_path, 0],
    points[hull_path, 1],
    color="red",
    linewidth=1,
    ls="--",
)

# Plot cutout region
cutout_center = (53.13, -28.10)
cutout_r = 0.5  # degrees
theta = np.linspace(0, 2 * np.pi, 100)
cutout = cutout_r * np.column_stack((np.cos(theta), np.sin(theta))) + cutout_center
ax1.plot(cutout[:, 0], cutout[:, 1], color="b", linewidth=1, ls="--")


# Histogram of i-band depth
cutout_mask = (combined.coord_ra - cutout_center[0]) ** 2 + (
    combined.coord_dec - cutout_center[1]
) ** 2 < cutout_r**2
settings = dict(bins=50, histtype="step", range=(23.5, 26.6))
ax2.hist(i5[cutout_mask], label="Inside cutout", **settings)
ax2.hist(i5[~cutout_mask], label="Outside cutout", **settings)
ax2.set(
    xlabel="$i$-band 5$\\sigma$ depth",
    ylabel="Number of objects",
    xlim=settings["range"],
)
ax2.legend(fontsize=8, frameon=False)

fig.subplots_adjust(wspace=0.55)
fig.savefig("../figures/dp1_footprint.pdf", bbox_inches="tight")

cutout = combined[cutout_mask]

## Compute colors and SNRs

In [ ]:
final = cutout.copy()

final["u-g"] = final.u_gaap1p0Mag - final.g_gaap1p0Mag
final["u-g_err"] = np.sqrt(final.u_gaap1p0MagErr**2 + final.g_gaap1p0MagErr**2)

final["g-r"] = final.g_gaap1p0Mag - final.r_gaap1p0Mag
final["g-r_err"] = np.sqrt(final.g_gaap1p0MagErr**2 + final.r_gaap1p0MagErr**2)

final["r-i"] = final.r_gaap1p0Mag - final.i_gaap1p0Mag
final["r-i_err"] = np.sqrt(final.r_gaap1p0MagErr**2 + final.i_gaap1p0MagErr**2)

final["i-z"] = final.i_gaap1p0Mag - final.z_gaap1p0Mag
final["i-z_err"] = np.sqrt(final.i_gaap1p0MagErr**2 + final.z_gaap1p0MagErr**2)

final["z-y"] = final.z_gaap1p0Mag - final.y_gaap1p0Mag
final["z-y_err"] = np.sqrt(final.z_gaap1p0MagErr**2 + final.y_gaap1p0MagErr**2)

final["u_snr"] = final.u_cModelFlux / final.u_cModelFluxErr
final["g_snr"] = final.g_cModelFlux / final.g_cModelFluxErr
final["r_snr"] = final.r_cModelFlux / final.r_cModelFluxErr
final["i_snr"] = final.i_cModelFlux / final.i_cModelFluxErr
final["z_snr"] = final.z_cModelFlux / final.z_cModelFluxErr
final["y_snr"] = final.y_cModelFlux / final.y_cModelFluxErr

final.to_parquet("../data/dp1_processed_full.pq")